In [ ]:
#######
# This shows the mpg.csv dataset as a spread out scatter plot
# without any callbacks.
######
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output
import plotly.graph_objs as go
import pandas as pd
from numpy import random

app = dash.Dash()

df = pd.read_csv('../data/mpg.csv')
# Add a random "jitter" to model_year to spread out the plot
df['year'] = random.randint(-4,5,len(df))*0.10 + df['model_year']

app.layout = html.Div([
    html.Div([   # this Div contains our scatter plot
    dcc.Graph(
        id='mpg_scatter',
        figure={
            'data': [go.Scattergl(
                x = df['year']+1900,  # our "jittered" data
                y = df['mpg'],
                text = df['name'],
                hoverinfo = 'text',
                mode = 'markers'
            )],
            'layout': go.Layout(
                title = 'mpg.csv dataset',
                xaxis = {'title': 'model year'},
                yaxis = {'title': 'miles per gallon'},
                hovermode='closest'
            )
        }
    )], style={'width':'50%','display':'inline-block'}),
    html.Div([  # this Div contains our output graph and vehicle stats
    dcc.Graph(
        id='mpg_line',
        figure={
            'data': [go.Scattergl(
                x = [0,1],
                y = [0,1],
                mode = 'lines'
            )],
            'layout': go.Layout(
                title = 'acceleration',
                margin = {'l':0}
            )
        }
    ),
    dcc.Markdown(
        id='mpg_stats'
    )
    ], style={'width':'20%', 'height':'50%','display':'inline-block'})
])

In [ ]:
@app.callback(Output('mpg_line','figure'),
              [Input('mpg_scatter','hoverData')])
def callback_graph(hoverData):

    v_index = hoverData['points'][0]['pointIndex']
    figure = {'data':[go.Scattergl(x=[0,1],y=[0,60/df.iloc[v_index]['acceleration']],
                                 mode='lines',line={'width':2*df.iloc[v_index]['cylinders']})],
              'layout':go.Layout(title=df.iloc[v_index]['name'],
                                 xaxis={'visible':False},yaxis={'visible':False,'range':[0,60/df['acceleration'].min()]},
                                 margin={'l':0},height=300)}
    
    return(figure)

In [ ]:
@app.callback(Output('mpg_stats','children'),
              [Input('mpg_scatter','hoverData')])
def callback_stats(hoverData):

    v_index = hoverData['points'][0]['pointIndex']
    stats = """
            {} cylinders\n
            {}cc displacement\n
            0 to 60mph in {} seconds\n
        """.format(df.iloc[v_index]['cylinders'],
            df.iloc[v_index]['displacement'],
            df.iloc[v_index]['acceleration'])
    
    return(stats)

In [ ]:
app.run_server(host='0.0.0.0')